In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [6]:
from keras.layers import Dense, Input, Flatten
from keras.layers import Reshape, Dropout, Concatenate
from keras.layers import Conv2D, MaxPool2D, Embedding
from keras.models import Model

In [7]:
dir = ""
glove = ""
text = ""

In [8]:
max_sequence_length = 50
max_nb_word = 20000
embedding_dim = 100
validation_split = 0.1

In [12]:
text = np.load(open("data/train_text.npy", 'rb'), allow_pickle=True).tolist()
label = np.load(open("data/train_label.npy", 'rb'), allow_pickle=True).tolist()

In [14]:
test_text = np.load(open("data/test_text.npy", 'rb'), allow_pickle=True).tolist()
test_label = np.load(open("data/test_label.npy", 'rb'), allow_pickle=True).tolist()

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(label)

LabelEncoder()

In [18]:
import joblib
joblib.dump(le, "data/label_encoder.pkl")

['data/label_encoder.pkl']

In [ ]:
label = le.transform(label)
test_label = le.transform(test_label)

In [21]:
le.classes_

array(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'RateBook',
       'SearchCreativeWork', 'SearchScreeningEvent'], dtype='<U20')

In [22]:
label_dict = dict(zip(list(le.classes_), le.transform(list(le.classes_))))

In [23]:
label_dict

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'RateBook': 3,
 'SearchCreativeWork': 4,
 'SearchScreeningEvent': 5}

In [24]:
tokenizer = Tokenizer(num_words=max_nb_word)
tokenizer.fit_on_texts(text)

In [25]:
sequences = tokenizer.texts_to_sequences(text)

In [26]:
sequences

[[11, 191, 61, 4, 1, 4013, 1141, 1572, 15],
 [11, 2624, 1573, 3, 14, 15, 939, 82, 256, 188, 548],
 [11, 187, 42, 2625, 4014, 4, 1968],
 [11, 2626, 22, 2627, 4, 14, 192, 27],
 [11, 92, 4, 14, 651, 520, 195],
 [11, 1, 428, 119, 4, 14, 4015, 1142],
 [11, 726, 1969, 2628, 4, 849, 1, 58],
 [11, 1, 4016, 6, 449, 1970, 3, 196, 355, 470, 14, 15],
 [11, 1, 257, 471, 3, 1, 371, 4, 1, 4017, 109, 196, 1036],
 [11, 43, 1971, 727],
 [11, 4018, 4, 1, 264, 199, 601, 15],
 [11, 409, 1574, 4, 1, 2629, 15],
 [11, 43, 4, 14, 940, 652],
 [8, 46, 191, 104, 3, 1, 109, 196, 1036, 15],
 [11, 4, 15, 8, 125, 14, 1143, 230, 1, 257, 123, 2630, 10, 6, 850],
 [11, 1, 271, 22, 1, 342, 4, 14, 651, 520, 195, 15],
 [11, 4019, 2631, 4, 14, 203, 6, 210, 98],
 [11, 851, 6, 4020, 4021, 4, 1, 15, 258, 4022, 1141, 4023],
 [11, 1144, 4024, 4, 14, 1575, 1321, 281, 15],
 [8, 46, 191, 81, 3, 14, 1972, 602, 1973, 15],
 [11, 2632, 2633, 2634, 4, 14, 16, 9, 603, 15],
 [11, 1037, 4, 2635, 40, 2636, 308, 410, 941, 15],
 [11, 61, 4, 40

In [27]:
word_index = tokenizer.word_index

In [31]:
word_index

{'the': 1,
 'a': 2,
 'in': 3,
 'to': 4,
 'for': 5,
 'of': 6,
 'at': 7,
 'i': 8,
 'is': 9,
 'book': 10,
 'add': 11,
 'what': 12,
 'find': 13,
 'my': 14,
 'playlist': 15,
 'this': 16,
 '6': 17,
 'movie': 18,
 'me': 19,
 'rate': 20,
 'weather': 21,
 'and': 22,
 'be': 23,
 'show': 24,
 'give': 25,
 'restaurant': 26,
 'out': 27,
 'will': 28,
 'it': 29,
 'on': 30,
 'current': 31,
 'stars': 32,
 'forecast': 33,
 'points': 34,
 'playing': 35,
 'can': 36,
 'table': 37,
 'want': 38,
 'like': 39,
 "'s": 40,
 'movies': 41,
 'from': 42,
 'album': 43,
 'now': 44,
 'that': 45,
 'need': 46,
 '1': 47,
 'called': 48,
 'one': 49,
 'schedule': 50,
 'are': 51,
 'theatres': 52,
 'with': 53,
 'you': 54,
 'please': 55,
 "'d": 56,
 '2': 57,
 'house': 58,
 'four': 59,
 'time': 60,
 'song': 61,
 'reservation': 62,
 'by': 63,
 '4': 64,
 '3': 65,
 'where': 66,
 'novel': 67,
 'get': 68,
 'rating': 69,
 'put': 70,
 'two': 71,
 'three': 72,
 '5': 73,
 'people': 74,
 'animated': 75,
 'five': 76,
 'times': 77,
 'schedu

In [32]:
len(word_index)

9869

In [33]:
data = pad_sequences(sequences, maxlen=max_sequence_length)

In [34]:
data

array([[   0,    0,    0, ..., 1141, 1572,   15],
       [   0,    0,    0, ...,  256,  188,  548],
       [   0,    0,    0, ..., 4014,    4, 1968],
       ...,
       [   0,    0,    0, ..., 9869,  929,  103],
       [   0,    0,    0, ...,   53,   18,   77],
       [   0,    0,    0, ...,    1,  105,   41]])

In [36]:
labels = to_categorical(np.asarray(label))

In [37]:
labels

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [39]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]

labels = labels[indices]
num_validation_samples = int(validation_split * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

Preparing embedding matrix.


In [41]:
# Specifically, we will use the 100-dimensional GloVe embeddings of 400k words computed on a 2014 dump of English Wikipedia. You can download them from http://nlp.stanford.edu/data/glove.6B.zip and extract it under the directory, BASE_DIR + '/glove.6B/'

In [ ]:
# fixxxx
embeddings_index = {}
f = open("glove.6B/", encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [48]:
filter_sizes = [2,3,5]
num_filters = 512
drop = 0.5

print("Creating Model...")
inputs = Input(shape=(max_sequence_length,), dtype='int32')
embedding = Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, weights=[embedding_matrix],
                      input_length=MAX_SEQUENCE_LENGTH, trainable=False)(inputs)
reshape = Reshape((max_sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(max_sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(max_sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(max_sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
preds = Dense(len(label_dict), activation='softmax')(dropout)

# this creates a model that includes inputs and outputs
model = Model(inputs=inputs, outputs=preds)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

Creating Model...


NameError: name 'embedding_matrix' is not defined

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          validation_data=(x_val, y_val))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_input = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Get predictions
test_predictions_probas = model.predict(test_input)
test_predictions = test_predictions_probas.argmax(axis=-1)

In [ ]:
from sklearn.metrics import classification_report

test_intent_predictions = train_le.inverse_transform(test_predictions)
test_intent_original = train_le.inverse_transform(test_labels)

print('accuracy: ', sum(test_intent_predictions == test_intent_original) / len(test_labels))
print("Precision, Recall and F1-Score:\n\n", classification_report(test_intent_original, test_intent_predictions))